In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.seasonal import seasonal_decompose

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
azeri_light = pd.read_excel('data/decomposition/azeri_light.xlsx')
brent = pd.read_excel('data/decomposition/brent.xls')
gdp = pd.read_csv('data/decomposition/gdp.csv')
loan_history = pd.read_csv('data/decomposition/loan_history.csv')


In [ ]:
loan_history.rename(columns={'T_DMCREDITSUM':'loan_amount'},inplace=True)
loan_history['T_BDBEGINDATE'] = pd.to_datetime(loan_history['T_BDBEGINDATE'])
loan_history['month'] = loan_history['T_BDBEGINDATE'].dt.to_period('M')
loan_history['quarter'] = loan_history['T_BDBEGINDATE'].dt.to_period('Q')
loan_history['year'] = loan_history['T_BDBEGINDATE'].dt.year
loan_history = loan_history.query("year>=2020").sort_values('T_BDBEGINDATE')
loan_history = loan_history.groupby('quarter',as_index=False)['loan_amount'].sum()

azeri_light['Day'] = pd.to_datetime(azeri_light['Day'])
azeri_light['year']=azeri_light['Day'].dt.year
azeri_light['quarter'] = azeri_light['Day'].dt.to_period('Q')
azeri_light = azeri_light.query("year>=2020").sort_values('Day')

brent['Date'] = pd.to_datetime(brent['Date'])
brent['year'] = brent['Date'].dt.year
brent['quarter'] = brent['Date'].dt.to_period('Q')
brent = brent.query("year>=2020").sort_values('Date')

gdp.rename(columns={'Year':'year'},inplace=True)


azeri_q = azeri_light[['quarter', 'Azeri_Light']].rename(columns={'Azeri_Light': 'amount'})
brent_q = brent[['quarter', 'price']].rename(columns={'price': 'amount'}).copy()
loan_q = loan_history[['quarter', 'loan_amount']].rename(columns={'loan_amount': 'amount'})


In [ ]:
# Merge quarterly dataframes: aggregate azeri and brent by quarter (mean), keep loan amounts, then merge
azeri_q_agg = azeri_q.groupby('quarter', as_index=False)['amount'].mean().rename(columns={'amount': 'azeri_avg'})
brent_q_agg = brent_q.groupby('quarter', as_index=False)['amount'].mean().rename(columns={'amount': 'brent_avg'})
loan_q_agg = loan_q.rename(columns={'amount': 'loan_amount'})

merged_quarterly = azeri_q_agg.merge(brent_q_agg, on='quarter', how='outer') \
                              .merge(loan_q_agg, on='quarter', how='outer') \
                              .sort_values('quarter') \
                              .reset_index(drop=True)

merged_quarterly

In [ ]:
# visualize merged_quarterly: lines for azeri_avg & brent_avg, secondary axis bars for loan_amount
df_plot = merged_quarterly.copy()
df_plot['quarter_start'] = df_plot['quarter'].dt.to_timestamp()

import matplotlib.ticker as ticker

fig, ax1 = plt.subplots(figsize=(12,5))

ax1.plot(df_plot['quarter_start'], df_plot['azeri_avg'],
         marker='o', label='Azeri Light (avg)', color='tab:blue')
ax1.plot(df_plot['quarter_start'], df_plot['brent_avg'],
         marker='s', label='Brent (avg)', color='tab:orange')
ax1.set_xlabel('Quarter')
ax1.set_ylabel('Price (USD)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
# show bars only where loan_amount is present
ax2.bar(df_plot['quarter_start'], df_plot['loan_amount'].fillna(0),
        width=70, alpha=0.25, color='tab:green', label='Loan Amount')
ax2.set_ylabel('Loan Amount (AZN)', color='tab:green')
ax2.tick_params(axis='y', labelcolor='tab:green')
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x/1e9:.1f}B'))

# legend, formatting
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')

fig.autofmt_xdate()
plt.title('Quarterly: Azeri Light & Brent (avg) and Loan Amount')
plt.grid(alpha=0.2)
plt.tight_layout()
plt.show()

In [ ]:
df = merged_quarterly[['quarter','azeri_avg','loan_amount']]
index = df['quarter'].dt.to_timestamp()
data = df.set_index(index)['loan_amount']
decomposition = seasonal_decompose(data, model='additive', period=4)
fig = decomposition.plot()


In [ ]:
model = ETSModel(data, error='add', trend='add', seasonal='add', seasonal_periods=4)
fit = model.fit()
fit.summary()

In [ ]:
forecast = fit.forecast(steps=5)
forecast_index = pd.date_range(start=data.index[-1] + pd.offsets.QuarterBegin(), periods=5, freq='QE')
forecast_series = pd.Series(forecast.values, index=forecast_index)
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(data.index, data, label='Observed', marker='o')
ax.plot(forecast_series.index, forecast_series, label='Forecast', marker='o', linestyle='--')
ax.set_xlabel('Quarter')
ax.set_ylabel('Loan Amount (AZN)')
ax.set_title('Loan Amount Forecast for Next 5 Quarters')
ax.legend()
plt.show()

forecast_table = (
    forecast_series.rename('forecast_loan_amount').round(2)
    .reset_index().rename(columns={'index': 'quarter'})
)
forecast_table['quarter'] = forecast_table['quarter'].dt.to_period('Q')
forecast_table

In [ ]:
fitted = fit.fittedvalues.reindex(data.index)
residuals = data - fitted

data_nonzero = data.replace(0, pd.NA)
mape = (residuals.abs().div(data_nonzero)).dropna().mean() * 100
mpe = (residuals.div(data_nonzero)).dropna().mean() * 100
smape = (200 * residuals.abs().div(data.abs() + fitted.abs())).dropna().mean()

metrics_df = pd.DataFrame(
    {
        'MAE': residuals.abs().mean(),
        'RMSE': (residuals.pow(2).mean()) ** 0.5,
        'MAPE (%)': mape,
        'MPE (%)': mpe,
        'sMAPE (%)': smape,
    },
    index=['ETSModel']
).round(2)

metrics_df